In [1]:
# For inline plotting
%matplotlib inline

# For auto reloading
%load_ext autoreload
%autoreload 2


In [2]:
import pickle
from pathlib import Path

from mab import GleapAgent, GleapEpsilonGreedyAgent, UcbAgent, UniformlyRandomAgent
import simulation_config


# Load all the simulation objects
# date = "20210831_150655"
# date = "20210901_100931"
# date = "20220126_233658"
# date = "20220127_150622"
date = "20220127_223730"
simulation_list = []
pathlist = Path("./simulations_data/gamma/{:s}".format(date)).glob('*.pkl')
for path in pathlist:
    with open(path, "rb") as f:
        simulation_list.append(pickle.load(f))


# Sort the simulation list
def value_func(simulation_index):
    simulation = simulation_list[simulation_index]
    agent = simulation.agent
    
    # Assume that the parameters are smaller than max_param
    max_param = 1000000.0
    value = 0.0
    if isinstance(agent, GleapAgent):
        value += 4 * max_param
        value += agent.gamma
        
        agent.name = "$\gamma = {:.2f}$".format(agent.gamma)
        agent.name_bold = "$\mathbf{{\gamma = {:.2f}}}$".format(agent.gamma)
    elif isinstance(agent, GleapEpsilonGreedyAgent):
        value += 3 * max_param
        value += agent.epsilon
    elif isinstance(agent, UcbAgent):
        value += 2 * max_param
        value += agent.gamma
    elif isinstance(agent, UniformlyRandomAgent):
        value += 1 * max_param
        
        agent.name = "Random"
    
    return value


simulation_index_list = [ i for i in range(len(simulation_list)) ]
sorted_simulation_index_list = sorted(simulation_index_list, key=lambda i: value_func(i))
for simulation_index in sorted_simulation_index_list:
    simulation = simulation_list[simulation_index]
    print(simulation.agent.name)


$\gamma = 0.00$
$\gamma = 0.50$
$\gamma = 1.00$
$\gamma = 2.00$
$\gamma = 4.00$


In [3]:
import math
from pathlib import Path
import shutil

import numpy as np
import matplotlib
print("matplotlib.__version__:", matplotlib.__version__)
print('Default backend: ' + matplotlib.get_backend())
matplotlib.use("module://mplcairo.base")
print('Backend is now ' + matplotlib.get_backend())
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from mab import GleapAgent, GleapEpsilonGreedyAgent


def calculate_metrics(simulation_index):
    simulation = simulation_list[simulation_index]
    
    # For reward
    time_averaged_compound_reward_array = np.mean(simulation.time_averaged_compound_reward_2darray, axis=0)
    aggregated_compound_reward_array = np.mean(simulation.aggregated_compound_reward_2darray, axis=0)
    
    # For regret
    time_averaged_regret_array = np.mean(simulation.time_averaged_regret_2darray, axis=0)
    cumulative_regret_array = np.mean(simulation.cumulative_regret_2darray, axis=0)
    
    # For time-averaged total prediction accuracy
    time_averaged_total_prediction_accuracy_array = np.mean(simulation.time_averaged_total_prediction_accuracy_2darray, axis=0)
    
    # For queue length
    if isinstance(simulation.agent, (GleapAgent, GleapEpsilonGreedyAgent)):
        q_array = np.mean(simulation.q_2darray, axis=0)
    else:
        q_array = None
    
    # For time-averaged total prediction latency
    time_averaged_total_prediction_latency_array = np.mean(simulation.time_averaged_total_prediction_latency_2darray, axis=0)
    
    # For time-averaged energy consumption
    time_averaged_energy_consumption_array = np.mean(simulation.time_averaged_energy_consumption_2darray, axis=0)
    
    return {\
            "time_averaged_compound_reward_array": time_averaged_compound_reward_array,\
            "aggregated_compound_reward_array": aggregated_compound_reward_array,\
            "time_averaged_regret_array": time_averaged_regret_array,\
            "cumulative_regret_array": cumulative_regret_array,\
            "time_averaged_total_prediction_accuracy_array": time_averaged_total_prediction_accuracy_array,\
            "q_array": q_array,\
            "time_averaged_total_prediction_latency_array": time_averaged_total_prediction_latency_array,\
            "time_averaged_energy_consumption_array": time_averaged_energy_consumption_array\
           }


def plot():
    # mkdir
    figure_path = Path("./simulations_data/gamma/{:s}/figures".format(date))
    figure_path.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(figure_path)
    figure_path.mkdir(parents=True, exist_ok=True)
    chinese_figure_path = Path("./simulations_data/gamma/{:s}/chinese_figures".format(date))
    chinese_figure_path.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(chinese_figure_path)
    chinese_figure_path.mkdir(parents=True, exist_ok=True)
    
    
    color_list = [ '#009E73', '#0072B2', '#CC79A7', '#E69F00', '#D55E00' ]
    marker_list = [ 'D', 's', 'o', 'v', '*' ]
    num_round = simulation_list[0].num_round
    MARK_EVERY = (num_round // 10, num_round // 5)
    MARKER_SIZE = 25.0
    MARKER_SIZE_CHINESE = 14.0
    LINE_WIDTH = 6.0
    LINE_WIDTH_CHINESE = 3.0
    FONT_SIZE = 36.0
    FONT_SIZE_CHINESE = 32.0
    FONT_SIZE_TICK = 32.0
    FONT_SIZE_TICK_CHINESE = 26.0
    FONT_SIZE_TICK_OFFSET_TEXT = 28.0
    FONT_SIZE_TICK_OFFSET_TEXT_CHINESE = 24.0
    FIG_SIZE = (16,9)
    FIG_SIZE_CHINESE = (8,6)
    DPI = 120
    DPI_CHINESE = 300
    GRID = True
    GRID_LINE_STYLE = (0,(8,4))
    FONT_WEIGHT = "normal"
    CHINESE_FONT_WEIGHT = "normal"
    if CHINESE_FONT_WEIGHT == "bold":
        CHINESE_FP = fm.FontProperties(fname='./font/source_han_serif_sc_bold.otf', math_fontfamily="stix")
    else:
        CHINESE_FP = fm.FontProperties(fname='./font/source_han_serif_sc_regular.otf', math_fontfamily="stix")
    SAVE_CHINESE_PDF = True
    
    
    time_averaged_compound_reward_array_list = []
    aggregated_compound_reward_array_list = []
    time_averaged_regret_array_list = []
    cumulative_regret_array_list = []
    time_averaged_total_prediction_accuracy_array_list = []
    q_array_list = []
    time_averaged_total_prediction_latency_array_list = []
    time_averaged_energy_consumption_array_list = []
    for i in sorted_simulation_index_list:
        metrics = calculate_metrics(i)
        
        time_averaged_compound_reward_array_list.append(metrics["time_averaged_compound_reward_array"])
        aggregated_compound_reward_array_list.append(metrics["aggregated_compound_reward_array"])
        time_averaged_regret_array_list.append(metrics["time_averaged_regret_array"])
        cumulative_regret_array_list.append(metrics["cumulative_regret_array"])
        time_averaged_total_prediction_accuracy_array_list.append(metrics["time_averaged_total_prediction_accuracy_array"])
        q_array_list.append(metrics["q_array"])
        time_averaged_total_prediction_latency_array_list.append(metrics["time_averaged_total_prediction_latency_array"])
        time_averaged_energy_consumption_array_list.append(metrics["time_averaged_energy_consumption_array"])
    
    
    # Display the converged average reward
    print("$\gamma$, Average Reward")
    for index, simulation_index in enumerate(sorted_simulation_index_list):
        simulation = simulation_list[simulation_index]
        converged_time_averaged_compound_reward = time_averaged_compound_reward_array_list[index][-1]
        print("{:.2f}, {:.4f}".format(simulation.agent.gamma, converged_time_averaged_compound_reward))
        
    
    ##########################################################################################
    # Beginning for time-averaged compound reward
    
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
    plt.locator_params(axis='x', nbins=5)
    for index, time_averaged_compound_reward_array in enumerate(time_averaged_compound_reward_array_list):
        simulation_index = sorted_simulation_index_list[index]
        agent = simulation_list[simulation_index].agent
        ax.plot(range(1, time_averaged_compound_reward_array.shape[0] + 1), time_averaged_compound_reward_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
    plt.grid(GRID, linestyle=GRID_LINE_STYLE)
    ax.set_ylim(0.0, 0.5)
    legend = ax.legend(shadow=True, prop=fm.FontProperties(size=32.0, style=FONT_WEIGHT))
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0), useMathText=True)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.xaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT, fontweight=FONT_WEIGHT)
    plt.setp(ax.yaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT, fontweight=FONT_WEIGHT)
    ax.set_xlabel(r'Round', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_ylabel(r'Average Reward', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    # Save the figure
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_compound_reward.pdf'.format(date), format='pdf', bbox_inches='tight')
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_compound_reward.png'.format(date), format='png', bbox_inches='tight')
    
    # For the Chinese version
    fig, ax = plt.subplots(figsize=FIG_SIZE_CHINESE, dpi=DPI_CHINESE)
    plt.locator_params(axis='x', nbins=5)
    for index, time_averaged_compound_reward_array in enumerate(time_averaged_compound_reward_array_list):
        simulation_index = sorted_simulation_index_list[index]
        agent = simulation_list[simulation_index].agent
        if CHINESE_FONT_WEIGHT == "bold":
            label = agent.name_bold
        else:
            label = agent.name
        ax.plot(range(1, time_averaged_compound_reward_array.shape[0] + 1), time_averaged_compound_reward_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=label, markersize=MARKER_SIZE_CHINESE, linewidth=LINE_WIDTH_CHINESE)
    plt.grid(GRID, linestyle=GRID_LINE_STYLE)
    ax.set_ylim(0.0, 0.5)
    if CHINESE_FONT_WEIGHT == "bold":
        legend = ax.legend(shadow=False, ncol=2, prop=fm.FontProperties(size=20.0, style="normal", fname='./font/source_han_serif_sc_bold.otf', math_fontfamily="stix"))
    else:
        legend = ax.legend(shadow=False, ncol=2, prop=fm.FontProperties(size=20.0, style="normal", fname='./font/source_han_serif_sc_regular.otf', math_fontfamily="stix"))
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0), useMathText=True)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    plt.setp(ax.xaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    plt.setp(ax.yaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    ax.set_xlabel(r'轮次', fontsize=FONT_SIZE_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    ax.set_ylabel(r'平均奖励', fontsize=FONT_SIZE_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    # Save the figure
    fig.savefig('./simulations_data/gamma/{:s}/chinese_figures/fig_gamma_simulations_time_averaged_compound_reward.png'.format(date), format='png', bbox_inches='tight')
    if SAVE_CHINESE_PDF:
        fig.savefig('./simulations_data/gamma/{:s}/chinese_figures/fig_gamma_simulations_time_averaged_compound_reward.pdf'.format(date), format='pdf', bbox_inches='tight')
    
    # Ending for time-averaged compound reward
    ##########################################################################################
    

#     # For time-averaged regret
#     fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
#     plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
#     plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
#     ax.yaxis.get_offset_text().set_fontsize(20)
#     ax.yaxis.get_offset_text().set_fontweight(FONT_WEIGHT)
#     ax.xaxis.get_offset_text().set_fontsize(20)
#     ax.xaxis.get_offset_text().set_fontweight(FONT_WEIGHT)
# #     ax.set_yticks([ -0.25, -0.05, 0.15 ])
#     plt.locator_params(axis='x', nbins=3)
#     for index, time_averaged_regret_array in enumerate(time_averaged_regret_array_list):
#         simulation_index = sorted_simulation_index_list[index]
#         agent = simulation_list[simulation_index].agent
#         ax.plot(range(1, time_averaged_regret_array.shape[0] + 1), time_averaged_regret_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
# #     legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), shadow=True, ncol=2, fontsize=20.0)
#     legend = ax.legend(shadow=True, prop={ 'size': 26, 'weight': 'normal'})
#     plt.grid(GRID)
# #     plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
#     plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
#     ax.set_xlabel(r'Round ($\times 10^4$)', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
#     ax.set_ylabel(r'Average Regret', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
#     # Save the figure
#     fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_regret.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
#     fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_regret.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')
    
#     # For cumulative regret
#     fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
#     plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
#     plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
#     for index, cumulative_regret_array in enumerate(cumulative_regret_array_list):
#         simulation_index = sorted_simulation_index_list[index]
#         agent = simulation_list[simulation_index].agent
#         ax.plot(range(1, time_averaged_regret_array.shape[0] + 1), cumulative_regret_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
# #     legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), shadow=True, ncol=2, fontsize=20.0)
#     legend = ax.legend(shadow=True, prop={ 'size': 32, 'weight': 'bold'})
#     plt.grid(GRID)
#     plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
#     ax.set_xlabel(r'Round ($\mathbf{\times 10^4}$)', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
#     ax.set_ylabel(r'Cumulative Regret', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
#     # Save the figure
#     fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_cumulative_regret.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
#     fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_cumulative_regret.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')
    
    # For time-averaged total prediction accuracy
    fig, ax = plt.subplots(figsize=(10,6), dpi=200)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    for index, time_averaged_total_prediction_accuracy_array in enumerate(time_averaged_total_prediction_accuracy_array_list):
        simulation_index = sorted_simulation_index_list[index]
        agent = simulation_list[simulation_index].agent
        ax.plot(range(1, time_averaged_total_prediction_accuracy_array.shape[0] + 1), time_averaged_total_prediction_accuracy_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
#     legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), shadow=True, ncol=2, fontsize=20.0)
    legend = ax.legend(shadow=True, prop={ 'size': 32, 'weight': 'bold'})
    plt.grid(GRID, linestyle=GRID_LINE_STYLE)
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
    ax.set_xlabel(r'Round ($\mathbf{\times 10^4}$)', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_ylabel(r'Average Prediction Accuracy', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    # Save the figure
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_total_prediction_accuracy.pdf'.format(date), bbox_extra_artists=(legend,), format='pdf', bbox_inches='tight')
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_total_prediction_accuracy.png'.format(date), bbox_extra_artists=(legend,), format='png', bbox_inches='tight')
    
    # For queue length
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    ax.yaxis.get_offset_text().set_fontsize(20)
    ax.yaxis.get_offset_text().set_fontweight(FONT_WEIGHT)
    ax.xaxis.get_offset_text().set_fontsize(20)
    ax.xaxis.get_offset_text().set_fontweight(FONT_WEIGHT)
    ax.set_yticks([ 0, 200, 400 ])
#     ax.set_yticklabels(['0.0', '2.0', '4.0'], fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.locator_params(axis='x', nbins=3)
    for index, q_array in enumerate(q_array_list):
        if q_array is None:
            continue
        
        simulation_index = sorted_simulation_index_list[index]
        agent = simulation_list[simulation_index].agent
        ax.plot(range(q_array.shape[0]), q_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
#     legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), shadow=True, ncol=2, fontsize=20.0)
    legend = ax.legend(loc=(0.30, 0.58), shadow=True, ncol=2, prop={ 'size': 32, 'weight': 'bold'})
    plt.grid(GRID, linestyle=GRID_LINE_STYLE)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
    ax.set_xlabel(r'Round ($\mathbf{\times 10^4}$)', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_ylabel('Queue Length ($\\mathbf{\\times 10^2}$)\n', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
#     #########################
#     # For inset
#     axins = ax.inset_axes([0.10, 0.25, 0.45, 0.45])
#     for index, q_array in enumerate(q_array_list):
#         if q_array is None:
#             continue
        
#         simulation_index = sorted_simulation_index_list[index]
#         agent = simulation_list[simulation_index].agent
#         axins.plot(range(10, 2000 + 1), q_array[10:2000+1], marker_list[index], markevery=100, linestyle='-', color=color_list[index], label=agent.name, markersize=4.0)
#     x1, x2, y1, y2 = 10, 2000, 0.0, 20.0
#     axins.set_xlim(x1, x2)
#     axins.set_ylim(y1, y2)
# #     axins.set_xticklabels('')
# #     axins.set_yticklabels('')
#     ax.indicate_inset_zoom(axins)
#     #########################
    # Save the figure
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_queue_length.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_queue_length.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')
    
    # For time-averaged total prediction latency
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    for index, time_averaged_total_prediction_latency_array in enumerate(time_averaged_total_prediction_latency_array_list):
        simulation_index = sorted_simulation_index_list[index]
        agent = simulation_list[simulation_index].agent
        ax.plot(range(1, time_averaged_total_prediction_latency_array.shape[0] + 1), time_averaged_total_prediction_latency_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
#     legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), shadow=True, ncol=2, fontsize=20.0)
    legend = ax.legend(shadow=True, prop={ 'size': 32, 'weight': 'bold'})
    plt.grid(GRID, linestyle=GRID_LINE_STYLE)
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
    ax.set_xlabel(r'Round ($\mathbf{\times 10^4}$)', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_ylabel(r'Average Prediction Latency', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    # Save the figure
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_total_prediction_latency.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_total_prediction_latency.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')
    
    # For time-averaged energy consumption
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    for index, time_averaged_energy_consumption_array in enumerate(time_averaged_energy_consumption_array_list):
        simulation_index = sorted_simulation_index_list[index]
        agent = simulation_list[simulation_index].agent
        ax.plot(range(1, time_averaged_energy_consumption_array.shape[0] + 1), time_averaged_energy_consumption_array, marker_list[index], markevery=MARK_EVERY, linestyle='-', color=color_list[index], label=agent.name, markersize=MARKER_SIZE, linewidth=LINE_WIDTH)
    plt.hlines(simulation_list[0].env.b, 0, simulation_list[0].num_round + 2, colors='#f22613', linestyles='dashdot')
#     legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), shadow=True, ncol=2, fontsize=20.0)
    legend = ax.legend(shadow=True, prop={ 'size': 32, 'weight': 'bold'})
    plt.grid(GRID, linestyle=GRID_LINE_STYLE)
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
    ax.set_xlabel(r'Round ($\mathbf{\times 10^4}$)', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_ylabel(r'Average Energy Consumption', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    # Save the figure
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_energy_consumption.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
    fig.savefig('./simulations_data/gamma/{:s}/figures/fig_gamma_simulations_time_averaged_energy_consumption.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')


matplotlib.__version__: 3.5.1
Default backend: module://matplotlib_inline.backend_inline
Backend is now module://mplcairo.base


In [4]:
plot()


$\gamma$, Average Reward
0.00, 0.3526
0.50, 0.4314
1.00, 0.4284
2.00, 0.4249
4.00, 0.4141
